In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from cvxopt import matrix, solvers
from sklearn.metrics import accuracy_score

In [2]:
pickleFile = open("hw2_p3.pkl","rb")
data = pd.read_pickle(pickleFile)
#data

In [3]:
X_train = data['x_train']
x_test = data['x_test']
y_train = data['y_train']
y_test = data['y_test']

# 3.2

Part 1:

Radial basis function (RBF) kernels are a popular choice for kernel support vector machines (SVMs) for classification problems because they offer a number of advantages: Allows non-linear relationship, is more computationally efficient, has good robustness to noise, universal approximation, offers better performance, flexible in tuning, easy to interpret and understand. RBF kernels are a powerful and versatile tool for solving classification problems, especially those involving complex, non-linear data.

# 3.2

Part 2

In [ ]:
class CustomSVM:
    def __init__(self, C_param=1.0, kernel_type='rbf', gamma_param=0.1):
        self.C_param = C_param
        self.kernel_type = kernel_type
        self.gamma_param = gamma_param
        self.alpha_values = None
        self.bias_term = None
        self.support_vectors = None
        self.support_vector_labels = None

    def _custom_kernel(self, data1, data2):
        if self.kernel_type == 'rbf':
            return np.exp(-self.gamma_param * np.linalg.norm(data1 - data2)**2)

    def _construct_p_matrix(self, data, labels):
        n_samples, n_features = data.shape
        P_matrix = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                P_matrix[i, j] = labels[i] * labels[j] * self._custom_kernel(data[i], data[j])
        return P_matrix

    def fit(self, data, labels):
        n_samples, n_features = data.shape

        P_matrix = matrix(self._construct_p_matrix(data, labels))
        q_matrix = matrix(-np.ones(n_samples))
        G_matrix = matrix(np.vstack((-np.eye(n_samples), np.eye(n_samples))))
        h_vector = matrix(np.hstack((np.zeros(n_samples), self.C_param * np.ones(n_samples))))
        A_matrix = matrix(labels.reshape(1, -1), (1, n_samples), 'd')
        b_vector = matrix(0.0)
        solution = solvers.qp(P_matrix, q_matrix, G_matrix, h_vector, A_matrix, b_vector)
        self.alpha_values = np.array(solution['x']).flatten()
        sv_indices = np.where(self.alpha_values > 1e-5)[0]
        self.support_vectors = data[sv_indices]
        self.support_vector_labels = labels[sv_indices]
        self.bias_term = np.mean(
            [labels[i] - np.sum(self.alpha_values * labels * self._custom_kernel(data, data[i])) for i in sv_indices]
        )

    def predict(self, data_input):
        if self.alpha_values is None or self.bias_term is None:
            raise ValueError("Train model before making predictions.")

        n_samples_input = data_input.shape[0]
        predictions_output = np.zeros(n_samples_input)

        for i in range(n_samples_input):
            prediction = 0
            for j in range(len(self.support_vectors)):
                prediction += self.alpha_values[j] * self.support_vector_labels[j] * self._custom_kernel(self.support_vectors[j], data_input[i])
            predictions_output[i] = prediction + self.bias_term

        return np.sign(predictions_output)




# 3.2

Part 3

In [ ]:
class CustomMultiClassSVM:
    def __init__(self, C_param=1.0, kernel_type='rbf', gamma_param=0.1):
        self.C_param = C_param
        self.kernel_type = kernel_type
        self.gamma_param = gamma_param
        self.binary_classifiers = {}

    def fit(self, data_input, labels_input):
        unique_classes = np.unique(labels_input)

        for i, class_label in enumerate(unique_classes):
            binary_labels = np.where(labels_input == class_label, 1, -1)
            binary_svm = CustomSVM(C_param=self.C_param, kernel_type=self.kernel_type, gamma_param=self.gamma_param)
            binary_svm.fit(data_input, binary_labels)
            self.binary_classifiers[class_label] = binary_svm

    def predict(self, data_input):
        class_scores = np.zeros((data_input.shape[0], len(self.binary_classifiers)))

        for i, class_label in enumerate(self.binary_classifiers):
            binary_svm = self.binary_classifiers[class_label]
            class_scores[:, i] = binary_svm.predict_raw(data_input)

        # Assign the class with the highest score as the predicted class
        predictions_output = np.argmax(class_scores, axis=1)

        return predictions_output

class CustomSVM:
    def __init__(self, C_param=1.0, kernel_type='rbf', gamma_param=0.1):
        self.C_param = C_param
        self.kernel_type = kernel_type
        self.gamma_param = gamma_param
        self.alpha_values = None
        self.bias_term = None
        self.support_vectors = None
        self.support_vector_labels = None

    def _custom_kernel(self, data1, data2):
        if self.kernel_type == 'rbf':
            return np.exp(-self.gamma_param * np.linalg.norm(data1 - data2)**2)

    def fit(self, data_input, labels_input):
        n_samples_input, n_features_input = data_input.shape
        P_matrix = matrix(np.outer(labels_input, labels_input) *
                          np.array([[self._custom_kernel(data_input[i], data_input[j]) for j in range(n_samples_input)] for i in range(n_samples_input)]))
        q_matrix = matrix(-np.ones(n_samples_input))
        G_matrix = matrix(np.vstack((-np.eye(n_samples_input), np.eye(n_samples_input))))
        h_vector = matrix(np.hstack((np.zeros(n_samples_input), self.C_param * np.ones(n_samples_input))))
        A_matrix = matrix(labels_input.reshape(1, -1), (1, n_samples_input), 'd')
        b_vector = matrix(0.0)
        solution = solvers.qp(P_matrix, q_matrix, G_matrix, h_vector, A_matrix, b_vector)

        self.alpha_values = np.array(solution['x']).flatten()

        sv_indices = np.where(self.alpha_values > 1e-5)[0]
        self.support_vectors = data_input[sv_indices]
        self.support_vector_labels = labels_input[sv_indices]

        self.bias_term = np.mean(
            [labels_input[i] - np.sum(self.alpha_values * labels_input * self._custom_kernel(data_input, data_input[i])) for i in sv_indices]
        )

    def predict_raw(self, data_input):
        if self.alpha_values is None or self.bias_term is None:
            raise ValueError("Train model before making predictions.")

        n_samples_input = data_input.shape[0]
        predictions_output = np.zeros(n_samples_input)

        for i in range(n_samples_input):
            prediction = 0
            for j in range(len(self.support_vectors)):
                prediction += self.alpha_values[j] * self.support_vector_labels[j] * self._custom_kernel(self.support_vectors[j], data_input[i])
            predictions_output[i] = prediction + self.bias_term

        return predictions_output

# 3.2

Part 4


In [ ]:
class CustomSVM:
    def __init__(self, C_param=1.0, kernel_type='rbf', gamma_param=0.1, degree_param=3, coef0_param=0.0):
        self.C_param = C_param
        self.kernel_type = kernel_type
        self.gamma_param = gamma_param
        self.degree_param = degree_param
        self.coef0_param = coef0_param
        self.alpha_values = None
        self.bias_term = None
        self.support_vectors = None
        self.support_vector_labels = None

    def _custom_kernel(self, data1, data2):
        if self.kernel_type == 'linear':
            return np.dot(data1, data2)
        elif self.kernel_type == 'poly':
            return (np.dot(data1, data2) + self.coef0_param) ** self.degree_param
        elif self.kernel_type == 'rbf':
            return np.exp(-self.gamma_param * np.linalg.norm(data1 - data2)**2)

    def fit(self, data_input, labels_input):
        n_samples_input, n_features_input = data_input.shape

        P_matrix = matrix(np.outer(labels_input, labels_input) *
                          np.array([[self._custom_kernel(data_input[i], data_input[j]) for j in range(n_samples_input)] for i in range(n_samples_input)]))
        q_matrix = matrix(-np.ones(n_samples_input))
        G_matrix = matrix(np.vstack((-np.eye(n_samples_input), np.eye(n_samples_input))))
        h_vector = matrix(np.hstack((np.zeros(n_samples_input), self.C_param * np.ones(n_samples_input))))
        A_matrix = matrix(labels_input.reshape(1, -1), (1, n_samples_input), 'd')
        b_vector = matrix(0.0)
        solution = solvers.qp(P_matrix, q_matrix, G_matrix, h_vector, A_matrix, b_vector)

        self.alpha_values = np.array(solution['x']).flatten()

        sv_indices = np.where(self.alpha_values > 1e-5)[0]
        self.support_vectors = data_input[sv_indices]
        self.support_vector_labels = labels_input[sv_indices]

        self.bias_term = np.mean(
            [labels_input[i] - np.sum(self.alpha_values * labels_input * self._custom_kernel(data_input, data_input[i])) for i in sv_indices]
        )

    def predict_raw(self, data_input):
        if self.alpha_values is None or self.bias_term is None:
            raise ValueError("Train model before making predictions.")

        n_samples_input = data_input.shape[0]
        predictions_output = np.zeros(n_samples_input)

        for i in range(n_samples_input):
            prediction = 0
            for j in range(len(self.support_vectors)):
                prediction += self.alpha_values[j] * self.support_vector_labels[j] * self._custom_kernel(self.support_vectors[j], data_input[i])
            predictions_output[i] = prediction + self.bias_term

        return predictions_output


In [ ]:
class CustomMultiClassSVM:
    def __init__(self, C_param=1.0, kernel_type='rbf', gamma_param=0.1, degree_param=3, coef0_param=0.0):
        self.C_param = C_param
        self.kernel_type = kernel_type
        self.gamma_param = gamma_param
        self.degree_param = degree_param
        self.coef0_param = coef0_param
        self.classifiers = {}

    def fit(self, data_input, labels_input):
        unique_classes = np.unique(labels_input)

        for i, class_label in enumerate(unique_classes):
            binary_labels = np.where(labels_input == class_label, 1, -1)
            binary_svm = CustomSVM(C_param=self.C_param, kernel_type=self.kernel_type, gamma_param=self.gamma_param, degree_param=self.degree_param, coef0_param=self.coef0_param)
            binary_svm.fit(data_input, binary_labels)
            self.classifiers[class_label] = binary_svm

    def predict(self, data_input):
        class_scores = np.zeros((data_input.shape[0], len(self.classifiers)))

        for i, class_label in enumerate(self.classifiers):
            classifier = self.classifiers[class_label]
            class_scores[:, i] = classifier.predict_raw(data_input)

        predictions_output = np.argmax(class_scores, axis=1)

        return predictions_output


In [ ]:
# When kernel is RBF

from sklearn.metrics import accuracy_score
multiclass_svm_custom = CustomMultiClassSVM(C_param=1.0, kernel_type='rbf')
multiclass_svm_custom.fit(X_train, y_train)
predictions_custom = multiclass_svm_custom.predict(x_test)

# Calculate the test accuracy
test_accuracy = accuracy_score(y_test, predictions_custom)

print(f"Test Accuracy Obatined for radial basis function: {test_accuracy * 100:.2f}%")

     pcost       dcost       gap    pres   dres
 0: -4.1678e+01 -8.5546e+02  4e+03  2e+00  2e-15
 1: -2.1866e+01 -3.7379e+02  5e+02  1e-01  2e-15
 2: -2.6698e+01 -8.3107e+01  6e+01  1e-02  2e-15
 3: -3.3279e+01 -6.2754e+01  3e+01  5e-03  1e-15
 4: -3.7476e+01 -4.9921e+01  1e+01  2e-03  1e-15
 5: -3.9313e+01 -4.5343e+01  6e+00  7e-04  1e-15
 6: -4.0606e+01 -4.2492e+01  2e+00  1e-04  2e-15
 7: -4.1072e+01 -4.1613e+01  5e-01  2e-05  2e-15
 8: -4.1240e+01 -4.1349e+01  1e-01  1e-06  2e-15
 9: -4.1265e+01 -4.1316e+01  5e-02  9e-08  2e-15
10: -4.1285e+01 -4.1292e+01  6e-03  9e-09  2e-15
11: -4.1288e+01 -4.1288e+01  1e-04  1e-10  2e-15
12: -4.1288e+01 -4.1288e+01  2e-06  2e-12  2e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.8454e+01 -8.1647e+02  3e+03  2e+00  2e-15
 1: -1.9536e+01 -3.4221e+02  4e+02  8e-02  2e-15
 2: -2.5789e+01 -8.4795e+01  7e+01  1e-02  2e-15
 3: -3.4007e+01 -5.4600e+01  2e+01  3e-03  2e-15
 4: -3.6769e+01 -4.6965e+01  1e+01  1e-03  2e-1

In [ ]:
# When Kernel is a linear function

from sklearn.metrics import accuracy_score
multiclass_svm_custom = CustomMultiClassSVM(C_param=1.0, kernel_type='linear')
multiclass_svm_custom.fit(X_train, y_train)
predictions_custom = multiclass_svm_custom.predict(x_test)

# Calculate the test accuracy
test_accuracy = accuracy_score(y_test, predictions_custom)

print(f"Test Accuracy Obtained for linear function: {test_accuracy * 100:.2f}%")

     pcost       dcost       gap    pres   dres
 0: -5.3311e+02 -1.3459e+03  4e+03  2e+00  1e-12
 1: -3.8082e+02 -8.7998e+02  5e+02  2e-14  2e-12
 2: -3.9923e+02 -4.0876e+02  1e+01  1e-14  1e-12
 3: -3.9999e+02 -4.0009e+02  1e-01  9e-14  1e-12
 4: -4.0000e+02 -4.0000e+02  1e-03  4e-14  1e-12
 5: -4.0000e+02 -4.0000e+02  1e-05  2e-14  1e-12
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -5.3333e+02 -1.3365e+03  4e+03  2e+00  1e-12
 1: -3.8146e+02 -8.7224e+02  5e+02  3e-14  2e-12
 2: -3.9928e+02 -4.0824e+02  9e+00  3e-14  2e-12
 3: -3.9999e+02 -4.0008e+02  9e-02  1e-13  2e-12
 4: -4.0000e+02 -4.0000e+02  9e-04  1e-14  1e-12
 5: -4.0000e+02 -4.0000e+02  9e-06  4e-14  1e-12
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -5.3316e+02 -1.3570e+03  4e+03  2e+00  1e-12
 1: -3.7999e+02 -8.8908e+02  5e+02  4e-14  1e-12
 2: -3.9898e+02 -4.1483e+02  2e+01  7e-15  1e-12
 3: -3.9999e+02 -4.0015e+02  2e-01  1e-13  1e-12
 4: -4.0000e+02 -4.0000e

In [ ]:
# When kernel is a poly function

from sklearn.metrics import accuracy_score
multiclass_svm_custom = CustomMultiClassSVM(C_param=1.0, kernel_type='poly')
multiclass_svm_custom.fit(X_train, y_train)
predictions_custom = multiclass_svm_custom.predict(x_test)

# Calculate the test accuracy
test_accuracy = accuracy_score(y_test, predictions_custom)

print(f"Test Accuracy Obatined for poly function: {test_accuracy * 100:.2f}%")

     pcost       dcost       gap    pres   dres
 0: -5.3288e+02 -1.3407e+03  4e+03  2e+00  1e-09
 1: -3.8106e+02 -8.7574e+02  5e+02  2e-13  1e-09
 2: -3.9926e+02 -4.0841e+02  9e+00  1e-14  1e-09
 3: -3.9999e+02 -4.0008e+02  9e-02  8e-14  2e-09
 4: -4.0000e+02 -4.0000e+02  9e-04  9e-14  1e-09
 5: -4.0000e+02 -4.0000e+02  1e-05  8e-15  1e-09
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -5.3317e+02 -1.3399e+03  4e+03  2e+00  2e-09
 1: -3.8121e+02 -8.7504e+02  5e+02  7e-14  2e-09
 2: -3.9927e+02 -4.0830e+02  9e+00  7e-14  2e-09
 3: -3.9999e+02 -4.0008e+02  9e-02  6e-15  2e-09
 4: -4.0000e+02 -4.0000e+02  9e-04  3e-14  2e-09
 5: -4.0000e+02 -4.0000e+02  1e-05  2e-13  2e-09
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -5.3223e+02 -1.4397e+03  5e+03  2e+00  3e-09
 1: -3.7841e+02 -9.7795e+02  7e+02  8e-02  3e-09
 2: -3.8470e+02 -5.5390e+02  2e+02  2e-02  2e-09
 3: -3.8941e+02 -4.3530e+02  5e+01  2e-03  2e-09
 4: -3.9495e+02 -4.0995e

RBF function offered the highest accuracy of 59.33% followed by poly at 36% and then linear function with 33.33%.